In [1]:
import pandas as pd
import numpy as np

from sklearn import svm

# train, test 데이터 자동 분리
from sklearn.model_selection import train_test_split
# classification에서 쓸 수 있게 0~1값으로 치환해줌
from sklearn.preprocessing import minmax_scale
# 칼럼에 있는 수많은 값들을 컴퓨터가 학습할 수 있는 숫자로 바꿔준다.
from sklearn.preprocessing import LabelEncoder
# 정답과 예측을 넣으면, 정확도를 산출해서 보여준다.
from sklearn.metrics import accuracy_score
# 평가지표를 한눈에 보게해주는 라이브러리
from sklearn.metrics import classification_report

In [2]:
csData = pd.read_csv("../dataset/customer.csv")
csData

,balance,stock,label
0,30000000,22500000,normal
1,280000000,48000000,diamond
2,300000000,40666666,diamond
3,54000000,28000000,normal
4,768000000,32000000,vip
...,...,...,...
19995,628000000,44666666,diamond
19996,276000000,20000000,normal
19997,652000000,41333333,diamond
19998,676000000,45333333,diamond


In [3]:
csData.balance.drop_duplicates()

0        30000000
1       280000000
2       300000000
3        54000000
4       768000000
          ...    
717     516000000
778     318000000
801     320000000
1239    508000000
5763    484000000
Name: balance, Length: 129, dtype: int64

In [4]:
csData.stock.drop_duplicates()

0       22500000
1       48000000
2       40666666
3       28000000
4       32000000
          ...   
1085    31000000
1378    39500000
1401    38500000
2494    39000000
5725    51000000
Name: stock, Length: 112, dtype: int64

In [5]:
csData.label.drop_duplicates()

0     normal
1    diamond
4        vip
Name: label, dtype: object

In [6]:
csData.label.isnull().sum()

0

In [7]:
csData.stock.isnull().sum()

0

In [8]:
csData.balance.isnull().sum()

0

In [9]:
# fit_transform은 숫자로 바꿔주고, inverse_transform은 원래대로 바꿔준다.
labelEn = LabelEncoder()
csData["labelcode"] = labelEn.fit_transform(csData.label)
csData

,balance,stock,label,labelcode
0,30000000,22500000,normal,1
1,280000000,48000000,diamond,0
2,300000000,40666666,diamond,0
3,54000000,28000000,normal,1
4,768000000,32000000,vip,2
...,...,...,...,...
19995,628000000,44666666,diamond,0
19996,276000000,20000000,normal,1
19997,652000000,41333333,diamond,0
19998,676000000,45333333,diamond,0


In [10]:
csData["labelcode"].drop_duplicates()

0    1
1    0
4    2
Name: labelcode, dtype: int32

In [11]:
csData["stock_norm"] = minmax_scale(csData.stock)
csData["balance_norm"] = minmax_scale(csData.balance)
csData

,balance,stock,label,labelcode,stock_norm,balance_norm
0,30000000,22500000,normal,1,0.080000,0.000000
1,280000000,48000000,diamond,0,0.488000,0.324675
2,300000000,40666666,diamond,0,0.370667,0.350649
3,54000000,28000000,normal,1,0.168000,0.031169
4,768000000,32000000,vip,2,0.232000,0.958442
...,...,...,...,...,...,...
19995,628000000,44666666,diamond,0,0.434667,0.776623
19996,276000000,20000000,normal,1,0.040000,0.319481
19997,652000000,41333333,diamond,0,0.381333,0.807792
19998,676000000,45333333,diamond,0,0.445333,0.838961


In [12]:
corrDf = csData.corr()
corrDf

,balance,stock,labelcode,stock_norm,balance_norm
balance,1.000000,0.565942,-0.253072,0.565942,1.000000
stock,0.565942,1.000000,0.339862,1.000000,0.565942
labelcode,-0.253072,0.339862,1.000000,0.339862,-0.253072
stock_norm,0.565942,1.000000,0.339862,1.000000,0.565942
balance_norm,1.000000,0.565942,-0.253072,0.565942,1.000000


In [25]:
stdRatio = 0.5
featCol = list(corrDf.loc[(abs(corrDf.labelcode)>stdRatio)&corrDf.labelcode!=1].index)
feature = featCol[2:]
label = ["label"]

In [26]:
labelCol = ["label"]

In [27]:
featuresData = csData.loc[:,features_Col]
labelData = csData.loc[:,labelCol]

In [28]:
print(featuresData.shape)
print(labelData.shape)

(20000, 0)
(20000, 1)


In [29]:
trainingData_features, \
testData_features, \
trainingData_label, \
testData_label = train_test_split(featuresData, labelData, test_size = 0.3, random_state = 1)

In [30]:
print(trainingData_features.shape)
print(trainingData_label.shape)
print(testData_features.shape)
print(testData_label.shape)

(14000, 0)
(14000, 1)
(6000, 0)
(6000, 1)


In [19]:
model_method = svm.SVC(random_state=1)
# svm = 선형분리

In [20]:
model = model_method.fit(X = trainingData_features,
                         y = trainingData_label)

ValueError: at least one array or dtype is required

In [ ]:
predict = model.predict(X = testData_features)

In [ ]:
predictData = pd.DataFrame(predict, columns = ["predict"])
predictData

In [ ]:
testData_label.reset_index(drop=True, inplace=True) 

In [ ]:
fianlData = pd.concat( [testData_label, predictData], axis=1)
fianlData

In [ ]:
labelData = csData.loc[testData_label.index,:]
labelData.reset_index(drop=True, inplace=True)
labelData

In [ ]:
finalResult = pd.concat( [labelData, predictData], axis=1) 
finalResult

In [ ]:
ac_score = accuracy_score(finalResult.label, finalResult.predict)
cl_report = classification_report(finalResult.label, finalResult.predict)

In [ ]:
print(ac_score)

In [ ]:
print(cl_report)